In [1]:
1+1

2

In [2]:
import pandas as pd
import os

In [3]:
%pwd

'/Users/suyash/Desktop/End to End/research'

In [4]:
os.chdir("../")
%pwd

'/Users/suyash/Desktop/End to End'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.IPL_Predictor.constants import *
from src.IPL_Predictor.utils.common import read_yaml, create_directories

In [7]:
import os 
import sys
from pathlib import Path
import urllib.request as request
import pandas as pd
from zipfile import ZipFile
import logging as logger
from src.IPL_Predictor.entity.config_entity import DataIngestionConfig
import zipfile


In [8]:
## component-Data Ingestion

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)


        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config

In [10]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-20 16:55:22,729: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-20 16:55:22,731: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-20 16:55:22,732: INFO: common: created directory at : artifacts]
[2025-09-20 16:55:22,733: INFO: common: created directory at : artifacts/data_ingestion]
[2025-09-20 16:55:22,734: INFO: 251335794: File already exists]
